In [27]:
# Load Library

packages <- c("dplyr", "haven","ggplot2","MASS", "randomForest", "caret", "ranger", "parallel")

install_if_missing <- function(pkg) {
  if (!requireNamespace(pkg, quietly = TRUE)) {
    install.packages(pkg)
  }
  library(pkg, character.only = TRUE)
}

lapply(packages, install_if_missing)

select <- dplyr::select

[[1]]
 [1] "parallel"     "ranger"       "caret"        "lattice"      "randomForest"
 [6] "MASS"         "ggplot2"      "haven"        "dplyr"        "stats"       
[11] "graphics"     "grDevices"    "utils"        "datasets"     "methods"     
[16] "base"        

[[2]]
 [1] "parallel"     "ranger"       "caret"        "lattice"      "randomForest"
 [6] "MASS"         "ggplot2"      "haven"        "dplyr"        "stats"       
[11] "graphics"     "grDevices"    "utils"        "datasets"     "methods"     
[16] "base"        

[[3]]
 [1] "parallel"     "ranger"       "caret"        "lattice"      "randomForest"
 [6] "MASS"         "ggplot2"      "haven"        "dplyr"        "stats"       
[11] "graphics"     "grDevices"    "utils"        "datasets"     "methods"     
[16] "base"        

[[4]]
 [1] "parallel"     "ranger"       "caret"        "lattice"      "randomForest"
 [6] "MASS"         "ggplot2"      "haven"        "dplyr"        "stats"       
[11] "graphics"     "grDevices"    "utils"        "datasets"     "methods"     
[16] "base"        

[[5]]
 [1] "parallel"     "ranger"       "caret"        "lattice"      "randomForest"
 [6] "MASS"         "ggplot2"      "haven"        "dplyr"        "stats"       
[11] "graphics"     "grDevices"    "utils"        "datasets"     "methods"     
[16] "base"        

[[6]]
 [1] "parallel"     "ranger"       "caret"        "lattice"      "randomForest"
 [6] "MASS"         "ggplot2"      "haven"        "dplyr"        "stats"       
[11] "graphics"     "grDevices"    "utils"        "datasets"     "methods"     
[16] "base"        

[[7]]
 [1] "parallel"     "ranger"       "caret"        "lattice"      "randomForest"
 [6] "MASS"         "ggplot2"      "haven"        "dplyr"        "stats"       
[11] "graphics"     "grDevices"    "utils"        "datasets"     "methods"     
[16] "base"        

[[8]]
 [1] "parallel"     "ranger"       "caret"        "lattice"      "randomForest"
 [6] "MASS"         "ggplot2"      "haven"        "dplyr"        "stats"       
[11] "graphics"     "grDevices"    "utils"        "datasets"     "methods"     
[16] "base"

# Maching Learning

In [34]:
df1 <- read.csv("Data/merge_df_cln.csv")
df1 <- df1 %>% 
        mutate(hyper = ifelse((bp_sys_mean >= 140 | bp_dia_mean >= 90), 1 , 0)) %>% 
        filter(htn_aware == "No") %>% #filter with Aware
        dplyr::select(-"X", -"bp_sys_mean", -"svy_id", -"bp_dia_mean", -"htn_aware")

colnames(df1)
# df1$hyper <- as.factor(df1$hyper)
# summary(df1$hyper)

[1] "svy_year"        "demo_age_cat"    "demo_race"       "demo_race_black"
 [5] "demo_age_years"  "demo_pregnant"   "demo_gender"     "cc_bmi"         
 [9] "DRXTPROT"        "DRXTCARB"        "DRXTTFAT"        "DRXTSFAT"       
[13] "DRXTMFAT"        "DRXTPFAT"        "DRXTCHOL"        "DRXTFIBE"       
[17] "DRXTVB6"         "DRXTVB12"        "DRXTFOLA"        "DRXTVC"         
[21] "DRXTCALC"        "DRXTPHOS"        "DRXTMAGN"        "DRXTIRON"       
[25] "DRXTZINC"        "DRXTCOPP"        "DRDTSODI"        "DRXTPOTA"       
[29] "DRXTSELE"        "DRXTCAFF"        "DRXTTHEO"        "DRXTALCO"       
[33] "DRXTMOIS"        "DRXTS040"        "DRXTS060"        "DRXTS080"       
[37] "DRXTS100"        "DRXTS120"        "DRXTS140"        "DRXTS160"       
[41] "DRXTS180"        "DRXTM161"        "DRXTM181"        "DRXTM201"       
[45] "DRXTM221"        "DRXTP182"        "DRXTP183"        "DRXTP184"       
[49] "DRXTP204"        "DRXTP205"        "DRXTP225"        "DRXTP226"       
[53] "DRD320GW"        "hyper"

## Random Forest

### Desrease size of majority

In [35]:
table(df1$hyper)

# Separate majority and minority classes
df1_majority <- df1 %>% filter(hyper == 0)
df1_minority <- df1 %>% filter(hyper == 1)

set.seed(42)
# Oversample minority class (duplicate existing data)
df1_majority_oversampled <- df1_majority %>% sample_n(nrow(df1_minority), replace = F)

# Combine balanced dataset
df1_balanced_decrease <- bind_rows(df1_minority, df1_majority_oversampled)

# Check new class distribution
table(df1_balanced_decrease$hyper)


    0     1 
29845  3680 


   0    1 
3680 3680 

In [36]:
# Change here for different resample method
df <- df1_balanced_decrease


# Split into training (70%) and testing (30%) sets
set.seed(42)
train_index <- createDataPartition(df$hyper, p = 0.7, list = FALSE)
train_data <- df[train_index, ]
test_data <- df[-train_index, ]

train_data$hyper <- as.factor(train_data$hyper)
test_data$hyper <- as.factor(test_data$hyper)

In [37]:
# Check available CPU cores
num_cores <- detectCores()
print(paste("Using", num_cores, "cores"))

# Train Random Forest classification model with multi-threading
rf_model <- ranger(hyper ~ ., data = train_data, 
                   num.trees = 500, 
                   mtry = sqrt(ncol(train_data) - 1), 
                   importance = "impurity",
                   probability = TRUE,  # Enables probability prediction
                   num.threads = num_cores)

# Predict class probabilities on the test set
predictions <- predict(rf_model, test_data)$predictions

# Convert probabilities to class labels
predicted_class <- ifelse(predictions[,2] > 0.5, "1", "0")  # Threshold at 0.5
predicted_class <- factor(predicted_class, levels = levels(test_data$hyper))

# Model Evaluation
conf_matrix <- confusionMatrix(predicted_class, test_data$hyper)

# Print Accuracy, Sensitivity, Specificity
print(conf_matrix)

# Variable Importance Plot
importance_values <- rf_model$variable.importance
sort(importance_values, decreasing = T)

[1] "Using 32 cores"
Confusion Matrix and Statistics

          Reference
Prediction   0   1
         0 754 253
         1 350 851
                                          
               Accuracy : 0.7269          
                 95% CI : (0.7078, 0.7454)
    No Information Rate : 0.5             
    P-Value [Acc > NIR] : < 2.2e-16       
                                          
                  Kappa : 0.4538          
                                          
 Mcnemar's Test P-Value : 9.252e-05       
                                          
            Sensitivity : 0.6830          
            Specificity : 0.7708          
         Pos Pred Value : 0.7488          
         Neg Pred Value : 0.7086          
             Prevalence : 0.5000          
         Detection Rate : 0.3415          
   Detection Prevalence : 0.4561          
      Balanced Accuracy : 0.7269          
                                          
       'Positive' Class : 0               
         

demo_age_years    demo_age_cat        DRXTMOIS        DRXTCARB        DRXTCALC 
     351.108904      240.962979       49.696176       43.719252       39.300271 
         DRXTVC        DRDTSODI        DRXTCAFF         DRXTVB6        DRXTPOTA 
      39.223848       38.839815       38.618285       38.537601       38.515220 
       DRXTFOLA        DRXTM201        DRXTCHOL        DRXTVB12        DRXTPROT 
      38.267965       38.192021       37.378188       37.225468       36.854727 
       DRXTS120        DRXTP204        DRXTIRON        DRXTSELE        DRXTZINC 
      36.662841       36.598173       36.270201       36.147135       36.105218 
       DRXTFIBE        DRXTP183        DRXTCOPP        DRXTPHOS        DRXTM161 
      35.723475       35.480631       35.416426       35.333891       35.043836 
       DRXTP182        DRXTM181        DRD320GW        DRXTPFAT        DRXTS140 
      35.005980       34.804971       34.663185       34.545867       34.258294 
       DRXTMAGN        DRXTMFAT          cc_bmi        DRXTS180        DRXTTFAT 
      34.040644       33.825261       33.200892       32.975236       32.927762 
       DRXTS080        DRXTS160        DRXTS040        DRXTSFAT        DRXTS100 
      32.707109       32.230404       31.736682       31.581750       31.065319 
       DRXTS060        DRXTM221        DRXTP226        DRXTP225        DRXTP205 
      29.292366       28.672196       26.963092       25.802190       24.353936 
       DRXTTHEO        svy_year        DRXTALCO     demo_gender        DRXTP184 
      23.466477       22.300750       20.996893       17.276316       15.741951 
      demo_race demo_race_black   demo_pregnant 
      12.744057        8.226591        0.000000

### Increase size of minority (Bootstrap)

In [38]:
table(df1$hyper)

# Separate majority and minority classes
df1_majority <- df1 %>% filter(hyper == 0)
df1_minority <- df1 %>% filter(hyper == 1)

set.seed(24)
# Oversample minority class (duplicate existing data)
df1_minority_oversampled <- df1_minority %>% sample_n(nrow(df1_majority), replace = T)

# Combine balanced dataset
df1_balanced_bootstrap <- bind_rows(df1_majority, df1_minority_oversampled)

# Check new class distribution
table(df1_balanced_bootstrap$hyper)


    0     1 
29845  3680 


    0     1 
29845 29845 

In [39]:
# Change here for different resample method
df <- df1_balanced_bootstrap


# Split into training (70%) and testing (30%) sets
set.seed(42)
train_index <- createDataPartition(df$hyper, p = 0.7, list = FALSE)
train_data <- df[train_index, ]
test_data <- df[-train_index, ]

train_data$hyper <- as.factor(train_data$hyper)
test_data$hyper <- as.factor(test_data$hyper)

In [40]:
# Check available CPU cores
num_cores <- detectCores()
print(paste("Using", num_cores, "cores"))

# Train Random Forest classification model with multi-threading
rf_model <- ranger(hyper ~ ., data = train_data, 
                   num.trees = 500, 
                   mtry = sqrt(ncol(train_data) - 1), 
                   importance = "impurity",
                   probability = TRUE,  # Enables probability prediction
                   num.threads = num_cores)

# Predict class probabilities on the test set
predictions <- predict(rf_model, test_data)$predictions

# Convert probabilities to class labels
predicted_class <- ifelse(predictions[,2] > 0.5, "1", "0")  # Threshold at 0.5
predicted_class <- factor(predicted_class, levels = levels(test_data$hyper))

# Model Evaluation
conf_matrix <- confusionMatrix(predicted_class, test_data$hyper)

# Print Accuracy, Sensitivity, Specificity
print(conf_matrix)

# Variable Importance Plot
importance_values <- rf_model$variable.importance
sort(importance_values, decreasing = T)

[1] "Using 32 cores"
Confusion Matrix and Statistics

          Reference
Prediction    0    1
         0 8587   42
         1  366 8911
                                          
               Accuracy : 0.9772          
                 95% CI : (0.9749, 0.9794)
    No Information Rate : 0.5             
    P-Value [Acc > NIR] : < 2.2e-16       
                                          
                  Kappa : 0.9544          
                                          
 Mcnemar's Test P-Value : < 2.2e-16       
                                          
            Sensitivity : 0.9591          
            Specificity : 0.9953          
         Pos Pred Value : 0.9951          
         Neg Pred Value : 0.9605          
             Prevalence : 0.5000          
         Detection Rate : 0.4796          
   Detection Prevalence : 0.4819          
      Balanced Accuracy : 0.9772          
                                          
       'Positive' Class : 0               
   

demo_age_years    demo_age_cat        DRXTMOIS        DRXTCARB        DRDTSODI 
     2907.15194      2129.98151       461.88215       389.60978       376.15928 
       DRXTCAFF        DRXTM201        DRXTFOLA          DRXTVC         DRXTVB6 
      363.95513       361.80278       361.11083       360.24186       353.33623 
       DRXTIRON        DRXTCALC        DRXTSELE        DRXTVB12        DRXTCHOL 
      352.45775       348.91503       347.24242       347.09478       345.60386 
       DRXTPOTA        DRXTCOPP        DRXTFIBE        DRXTM161        DRXTP183 
      343.34628       341.53366       340.71433       339.73281       334.82962 
       DRD320GW        DRXTPROT        DRXTP204        DRXTS120        DRXTZINC 
      332.34617       331.03937       329.69825       328.25522       326.46229 
       DRXTPHOS        DRXTMAGN          cc_bmi        DRXTPFAT        DRXTP182 
      325.82961       324.20442       321.17728       317.17492       316.58409 
       DRXTM181        DRXTMFAT        DRXTS180        DRXTS140        DRXTTFAT 
      307.17943       304.68294       302.22000       294.30666       292.61452 
       DRXTS080        DRXTS040        DRXTS160        DRXTS100        DRXTSFAT 
      290.05910       286.49208       286.13853       279.88913       267.21040 
       DRXTS060        DRXTM221        DRXTP226        DRXTP225        DRXTALCO 
      266.37420       254.29507       253.13222       235.32351       234.32940 
       DRXTP205        DRXTTHEO        svy_year     demo_gender        DRXTP184 
      233.91426       221.19929       204.83640       153.71445       151.66398 
      demo_race demo_race_black   demo_pregnant 
      122.42245        95.64153         0.00000

### Hybrid

In [74]:
# Step 1: Reduce the majority class (Under-sampling)
df1_majority <- df1 %>% filter(hyper == 0) %>% sample_n(nrow(df1_minority) * 1.5, replace = FALSE)

# Step 2: Increase the minority class (Over-sampling)
df1_minority_oversampled <- df1_minority %>% sample_n(nrow(df1_majority), replace = TRUE)

# Combine both
df1_balanced_hybrid <- bind_rows(df1_majority, df1_minority_oversampled)

# Check new class distribution
table(df1_balanced_hybrid$hyper)



   0    1 
5520 5520 

In [75]:
# Change here for different resample method
df <- df1_balanced_hybrid


# Split into training (70%) and testing (30%) sets
set.seed(24)
train_index <- createDataPartition(df$hyper, p = 0.7, list = FALSE)
train_data <- df[train_index, ]
test_data <- df[-train_index, ]

train_data$hyper <- as.factor(train_data$hyper)
test_data$hyper <- as.factor(test_data$hyper)

In [79]:
# Check available CPU cores
num_cores <- detectCores()
print(paste("Using", num_cores, "cores"))

# Train Random Forest classification model with multi-threading
rf_model <- ranger(hyper ~ ., data = train_data, 
                   num.trees = 1500, 
                   mtry = floor(sqrt(ncol(train_data) - 1) * 1.5), 
                   importance = "impurity",
                   probability = TRUE,
                   num.threads = parallel::detectCores())


# Predict class probabilities on the test set
predictions <- predict(rf_model, test_data)$predictions

# # Convert probabilities to class labels
# predicted_class <- ifelse(predictions[,2] > 0.5, "1", "0")  # Threshold at 0.5
# predicted_class <- factor(predicted_class, levels = levels(test_data$hyper))

# # Model Evaluation
# conf_matrix <- confusionMatrix(predicted_class, test_data$hyper)

# # Print Accuracy, Sensitivity, Specificity
# print(conf_matrix)

threshold <- 0.45  # Lower threshold slightly to increase sensitivity

predicted_class <- ifelse(predictions[,2] > threshold, "1", "0")  
predicted_class <- factor(predicted_class, levels = levels(test_data$hyper))

conf_matrix <- confusionMatrix(predicted_class, test_data$hyper)
print(conf_matrix)


# Variable Importance Plot
importance_values <- rf_model$variable.importance
sort(importance_values, decreasing = T)

[1] "Using 32 cores"
Confusion Matrix and Statistics

          Reference
Prediction    0    1
         0 1181  158
         1  475 1498
                                          
               Accuracy : 0.8089          
                 95% CI : (0.7951, 0.8221)
    No Information Rate : 0.5             
    P-Value [Acc > NIR] : < 2.2e-16       
                                          
                  Kappa : 0.6178          
                                          
 Mcnemar's Test P-Value : < 2.2e-16       
                                          
            Sensitivity : 0.7132          
            Specificity : 0.9046          
         Pos Pred Value : 0.8820          
         Neg Pred Value : 0.7592          
             Prevalence : 0.5000          
         Detection Rate : 0.3566          
   Detection Prevalence : 0.4043          
      Balanced Accuracy : 0.8089          
                                          
       'Positive' Class : 0               
   

demo_age_years    demo_age_cat        DRXTMOIS        DRXTCARB        DRXTFOLA 
      567.91731       367.82092        76.89415        69.69967        64.23250 
        DRXTVB6        DRDTSODI          DRXTVC        DRXTM201        DRXTIRON 
       64.18101        63.54745        62.80616        62.20194        61.77768 
       DRXTSELE        DRXTP204          cc_bmi        DRXTPROT        DRXTCHOL 
       61.63773        61.57586        60.61577        59.99718        59.73317 
       DRXTM161        DRXTPOTA        DRD320GW        DRXTVB12        DRXTCAFF 
       59.50475        58.58168        57.65271        57.64554        57.23187 
       DRXTP183        DRXTMAGN        DRXTCALC        DRXTZINC        DRXTFIBE 
       56.23965        56.21602        56.07804        55.49614        55.09849 
       DRXTCOPP        DRXTS120        DRXTP182        DRXTPHOS        DRXTPFAT 
       54.70640        51.53961        51.31831        51.27007        50.64821 
       DRXTM181        DRXTS180        DRXTMFAT        DRXTM221        DRXTS140 
       49.83063        48.99272        48.40162        48.26388        47.53258 
       DRXTS040        DRXTS080        DRXTS160        DRXTTFAT        DRXTS100 
       46.02303        45.69740        45.43275        45.04064        44.78874 
       DRXTS060        DRXTSFAT        DRXTALCO        DRXTP226        DRXTP225 
       44.15706        44.13747        43.92293        42.67011        41.17067 
       DRXTP205        DRXTTHEO     demo_gender        svy_year        DRXTP184 
       39.71727        35.87030        33.94916        33.77622        27.31529 
      demo_race demo_race_black   demo_pregnant 
       18.80864        14.09338         0.00000

In [29]:
# Change here for different resample method
df <- df1


# Split into training (70%) and testing (30%) sets
set.seed(42)
train_index <- createDataPartition(df$hyper, p = 0.7, list = FALSE)
train_data <- df[train_index, ]
test_data <- df[-train_index, ]

train_data$hyper <- as.factor(train_data$hyper)
test_data$hyper <- as.factor(test_data$hyper)

### Desrease size of majority

In [30]:
table(train_data$hyper)

# Separate majority and minority classes
train_data_majority <- train_data %>% filter(hyper == 0)
train_data_minority <- train_data %>% filter(hyper == 1)

set.seed(42)
# Oversample minority class (duplicate existing data)
train_data_majority_oversampled <- train_data_majority %>% sample_n(nrow(train_data_minority), replace = F)

# Combine balanced dataset
train_data_decrease <- bind_rows(train_data_minority, train_data_majority_oversampled)

# Check new class distribution
table(train_data_decrease$hyper)


    0     1 
28409  6986 


   0    1 
6986 6986 

In [31]:
# Check available CPU cores
num_cores <- detectCores()
print(paste("Using", num_cores, "cores"))

# Train Random Forest classification model with multi-threading
rf_model <- ranger(hyper ~ ., data = train_data_decrease, 
                   num.trees = 1000, 
                   mtry = sqrt(ncol(train_data_decrease) - 1), 
                   importance = "impurity",
                   probability = TRUE,  # Enables probability prediction
                   num.threads = num_cores)

# Predict class probabilities on the test set
predictions <- predict(rf_model, test_data)$predictions

# Convert probabilities to class labels
predicted_class <- ifelse(predictions[,2] > 0.5, "1", "0")  # Threshold at 0.5
predicted_class <- factor(predicted_class, levels = levels(test_data$hyper))

# Model Evaluation
conf_matrix <- confusionMatrix(predicted_class, test_data$hyper)

# Print Accuracy, Sensitivity, Specificity
print(conf_matrix)

# Variable Importance Plot
importance_values <- rf_model$variable.importance
sort(importance_values, decreasing = T)

[1] "Using 32 cores"
Confusion Matrix and Statistics

          Reference
Prediction    0    1
         0 7472  601
         1 4684 2411
                                          
               Accuracy : 0.6516          
                 95% CI : (0.6439, 0.6592)
    No Information Rate : 0.8014          
    P-Value [Acc > NIR] : 1               
                                          
                  Kappa : 0.275           
                                          
 Mcnemar's Test P-Value : <2e-16          
                                          
            Sensitivity : 0.6147          
            Specificity : 0.8005          
         Pos Pred Value : 0.9256          
         Neg Pred Value : 0.3398          
             Prevalence : 0.8014          
         Detection Rate : 0.4926          
   Detection Prevalence : 0.5322          
      Balanced Accuracy : 0.7076          
                                          
       'Positive' Class : 0               
   

demo_age_years    demo_age_cat        DRXTMOIS        DRXTCARB        DRXTCAFF 
      827.90050       600.12114       141.32242       131.33054       113.73029 
         DRXTVC        DRDTSODI          cc_bmi        DRXTFIBE        DRXTPOTA 
      112.98892       110.12139       108.18490       107.94053       107.08851 
       DRXTCHOL        DRXTCALC        DRXTCOPP        DRXTM161         DRXTVB6 
      106.79938       105.72871       105.58360       105.40654       104.12818 
       DRXTPHOS        DRXTFOLA        DRXTSELE        DRXTVB12        DRXTM201 
      104.03729       104.02293       103.66753       103.21787       102.80047 
       DRXTIRON        DRXTZINC        DRXTMAGN        DRXTP204        DRD320GW 
      101.61284       101.37363       100.86528        99.86277        99.34028 
       DRXTP183        DRXTPROT        DRXTS180        DRXTS120        DRXTP182 
       98.92800        98.46329        96.44993        96.42262        94.73904 
       DRXTMFAT        DRXTM181        DRXTPFAT        DRXTS160        DRXTSFAT 
       94.71661        93.93793        93.81192        93.75540        92.09933 
       DRXTS140        DRXTTFAT        DRXTS040        DRXTS080        DRXTS100 
       90.59576        89.97009        89.49686        87.87634        86.72619 
       DRXTS060        DRXTM221        DRXTP226        DRXTP205        DRXTP225 
       84.49258        77.91891        77.03815        71.14262        70.41229 
       DRXTTHEO        svy_year        DRXTALCO demo_race_black        DRXTP184 
       66.58474        63.12909        54.07222        45.57350        43.08608 
      demo_race     demo_gender   demo_pregnant 
       40.04647        23.72973         0.00000

### Increase size of minority (Bootstrap)

In [32]:
table(train_data$hyper)

# Separate majority and minority classes
train_data_majority <- train_data %>% filter(hyper == 0)
train_data_minority <- train_data %>% filter(hyper == 1)

set.seed(24)
# Oversample minority class (duplicate existing data)
train_data_minority_oversampled <- train_data_minority %>% sample_n(nrow(train_data_majority), replace = T)

# Combine balanced dataset
train_data_bootstrap <- bind_rows(train_data_majority, train_data_minority_oversampled)

# Check new class distribution
table(train_data_bootstrap$hyper)


    0     1 
28409  6986 


    0     1 
28409 28409 

In [33]:
# Check available CPU cores
num_cores <- detectCores()
print(paste("Using", num_cores, "cores"))

# Train Random Forest classification model with multi-threading
rf_model <- ranger(hyper ~ ., data = train_data_bootstrap, 
                   num.trees = 500, 
                   mtry = sqrt(ncol(train_data_bootstrap) - 1), 
                   importance = "impurity",
                   probability = TRUE,  # Enables probability prediction
                   num.threads = num_cores)

# Predict class probabilities on the test set
predictions <- predict(rf_model, test_data)$predictions

# Convert probabilities to class labels
predicted_class <- ifelse(predictions[,2] > 0.5, "1", "0")  # Threshold at 0.5
predicted_class <- factor(predicted_class, levels = levels(test_data$hyper))

# Model Evaluation
conf_matrix <- confusionMatrix(predicted_class, test_data$hyper)

# Print Accuracy, Sensitivity, Specificity
print(conf_matrix)

# Variable Importance Plot
importance_values <- rf_model$variable.importance
sort(importance_values, decreasing = T)

[1] "Using 32 cores"
Confusion Matrix and Statistics

          Reference
Prediction     0     1
         0 11355  2375
         1   801   637
                                         
               Accuracy : 0.7906         
                 95% CI : (0.784, 0.7971)
    No Information Rate : 0.8014         
    P-Value [Acc > NIR] : 0.9996         
                                         
                  Kappa : 0.1812         
                                         
 Mcnemar's Test P-Value : <2e-16         
                                         
            Sensitivity : 0.9341         
            Specificity : 0.2115         
         Pos Pred Value : 0.8270         
         Neg Pred Value : 0.4430         
             Prevalence : 0.8014         
         Detection Rate : 0.7486         
   Detection Prevalence : 0.9052         
      Balanced Accuracy : 0.5728         
                                         
       'Positive' Class : 0              
                 

demo_age_years    demo_age_cat        DRXTMOIS        DRXTCARB        DRXTCAFF 
      3481.9691       2425.2206        614.6591        578.5270        512.3409 
       DRDTSODI          DRXTVC        DRXTCALC        DRXTSELE        DRXTFOLA 
       509.3619        509.0074        496.3732        488.6282        487.2444 
       DRXTPOTA         DRXTVB6        DRXTCHOL        DRXTZINC        DRXTCOPP 
       486.3509        480.5206        479.7819        479.6807        478.7406 
       DRXTM201        DRXTFIBE        DRXTIRON        DRXTM161        DRXTVB12 
       477.3793        473.0090        472.5792        472.3993        471.2461 
       DRXTPHOS        DRXTMAGN        DRXTP183        DRXTPROT        DRXTP204 
       470.7141        468.4095        464.7071        464.1115        457.8371 
       DRXTS120        DRD320GW        DRXTS160        DRXTS180          cc_bmi 
       451.1087        443.3462        435.7084        435.5507        432.8923 
       DRXTP182        DRXTPFAT        DRXTS140        DRXTSFAT        DRXTMFAT 
       430.6582        425.5541        423.2306        418.4028        414.1384 
       DRXTM181        DRXTS040        DRXTTFAT        DRXTS100        DRXTS080 
       413.6530        410.9379        405.6895        405.6758        403.2113 
       DRXTS060        DRXTP226        DRXTM221        DRXTP205        DRXTP225 
       391.2662        357.9256        353.3837        328.7509        328.7335 
       DRXTTHEO        svy_year        DRXTALCO        DRXTP184 demo_race_black 
       306.4004        287.6743        224.6469        203.5947        182.0972 
      demo_race     demo_gender   demo_pregnant 
       179.6726        106.3170          0.0000

### Hybrid

In [24]:
# Step 1: Reduce the majority class (Under-sampling)
train_data_majority <- train_data %>% filter(hyper == 0) %>% sample_n(nrow(train_data_minority) * 1.5, replace = FALSE)

# Step 2: Increase the minority class (Over-sampling)
train_data_minority_oversampled <- train_data_minority %>% sample_n(nrow(train_data_majority), replace = TRUE)

# Combine both
train_data_hybrid <- bind_rows(train_data_majority, train_data_minority_oversampled)

# Check new class distribution
table(train_data_hybrid$hyper)



   0    1 
3913 3913 

In [26]:
# Check available CPU cores
num_cores <- detectCores()
print(paste("Using", num_cores, "cores"))

# Train Random Forest classification model with multi-threading
rf_model <- ranger(hyper ~ ., data = train_data_hybrid, 
                   num.trees = 1500, 
                   mtry = floor(sqrt(ncol(train_data_hybrid) - 1) * 1.5), 
                   importance = "impurity",
                   probability = TRUE,
                   num.threads = parallel::detectCores())


# Predict class probabilities on the test set
predictions <- predict(rf_model, test_data)$predictions

threshold <- 0.45  # Lower threshold slightly to increase sensitivity

predicted_class <- ifelse(predictions[,2] > threshold, "1", "0")  
predicted_class <- factor(predicted_class, levels = levels(test_data$hyper))

conf_matrix <- confusionMatrix(predicted_class, test_data$hyper)
print(conf_matrix)


# Variable Importance Plot
importance_values <- rf_model$variable.importance
sort(importance_values, decreasing = T)

[1] "Using 32 cores"
Confusion Matrix and Statistics

          Reference
Prediction    0    1
         0 6594  315
         1 2392  756
                                          
               Accuracy : 0.7308          
                 95% CI : (0.7221, 0.7395)
    No Information Rate : 0.8935          
    P-Value [Acc > NIR] : 1               
                                          
                  Kappa : 0.2371          
                                          
 Mcnemar's Test P-Value : <2e-16          
                                          
            Sensitivity : 0.7338          
            Specificity : 0.7059          
         Pos Pred Value : 0.9544          
         Neg Pred Value : 0.2402          
             Prevalence : 0.8935          
         Detection Rate : 0.6557          
   Detection Prevalence : 0.6870          
      Balanced Accuracy : 0.7198          
                                          
       'Positive' Class : 0               
   

demo_age_years    demo_age_cat        DRXTMOIS          cc_bmi        DRXTCARB 
      581.30780       383.25384        83.63974        75.93847        72.14065 
       DRDTSODI        DRXTCAFF        DRXTFOLA          DRXTVC        DRXTCALC 
       68.39863        66.62045        65.71525        65.35976        64.75043 
       DRD320GW        DRXTM201        DRXTFIBE        DRXTM161        DRXTPOTA 
       63.97481        62.34645        61.72225        61.26500        60.57544 
       DRXTSELE        DRXTMAGN         DRXTVB6        DRXTZINC        DRXTIRON 
       59.79938        59.35134        59.34851        58.74036        58.73596 
       DRXTCOPP        DRXTVB12        DRXTCHOL        DRXTP204        DRXTPHOS 
       57.95620        57.56017        56.47993        56.00806        54.97198 
       DRXTP183        DRXTPROT        DRXTS120        DRXTM181        DRXTPFAT 
       54.92103        54.24099        53.89365        51.15191        51.11813 
       DRXTP182        DRXTMFAT        DRXTM221        DRXTS180        DRXTS140 
       51.03273        50.94286        48.76247        48.72556        48.70614 
       DRXTS080        DRXTS040        DRXTS100        DRXTTFAT        DRXTS160 
       46.57046        46.37861        45.96169        45.39814        44.54945 
       DRXTSFAT        DRXTP225        DRXTS060        DRXTP226        DRXTP205 
       43.64755        43.08124        42.99350        42.82198        39.62024 
       DRXTALCO        DRXTTHEO        svy_year     demo_gender        DRXTP184 
       38.17040        35.79474        33.34986        27.86561        25.31320 
      demo_race demo_race_black   demo_pregnant 
       22.24152        14.13609         0.00000